In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os,shutil
import tensorflow

os.mkdir("./train")
shutil.copytree("../input/smiling-or-not-face-data/non_smile","./train/non_smile")
shutil.copytree("../input/smiling-or-not-face-data/smile","./train/smile")
from tensorflow.keras.preprocessing.image import ImageDataGenerator
generator =  ImageDataGenerator(rescale= 1/255,validation_split = 0.1)

train_data = generator.flow_from_directory(directory = "./train",
                                           target_size=(64,64),
                                           batch_size = 64,
                                          subset="training")
val_data = generator.flow_from_directory(directory = "./train",
                                         batch_size = 64,
                                         target_size=(64,64),
                                        subset = "validation")
generator = ImageDataGenerator(rescale= 1/255,validation_split = 0)
test_data = generator.flow_from_directory("../input/smiling-or-not-face-data/test",
                                         target_size=(64,64))



In [ ]:
from tensorflow import keras 
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
from tensorflow.keras.applications.densenet import DenseNet121


clfs = [VGG16,VGG19,ResNet50,DenseNet121]

for clf in clfs:
    print(clf)
    clf = clf(input_shape=(64,64,3), weights='imagenet', include_top=False)
    for layer in clf.layers:
        layer.trainable = False
    x = layers.Flatten()(clf.output)

    prediction = layers.Dense(2,activation = "softmax")(x)

    clf = Model(inputs=clf.input, outputs=prediction)

    clf.compile(optimizer='adam',
    loss=tensorflow.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.AUC(name='auc'),'acc'])
    callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=8,
                                                restore_best_weights=True)
    clf.fit(train_data, epochs=5, validation_data=val_data, callbacks=callback)
    loss,auc,acc = clf.evaluate(val_data)
    print(acc)